In [1]:
using AutomotiveDrivingModels
using AutoViz
using Reactive
using Interact

import PyPlot

/Users/kyle/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


# Types

In [2]:
type HRHC # Hierarchical Receding Horizon Controller
    #car parameters
    car_length::Float64
    car_width::Float64
    
    v::Float64
    δ::Float64
    
    h::Int
    Δt::Float64
    
    V_MIN::Float64
    V_MAX::Float64
    V_STEPS::Int

    δ_MAX::Float64
    δ_MIN::Float64
    δ_STEPS::Int64
    
    ΔV₊::Float64
    ΔV₋::Float64
    Δδ::Float64
    
    T_MAX::Float64
    obstacles
end

# functions

In [3]:
# Return a library of motion primitives (arcs of constant radius) representing short paths that the car can follow.
function MotionPrimitives(hrhc::HRHC)
    car_length=hrhc.car_length
    car_width=hrhc.car_width
    
    h=hrhc.h
    Δt=hrhc.Δt
    
    v_min = hrhc.V_MIN
    v_max = hrhc.V_MAX
    v_steps = hrhc.V_STEPS

    δ_max = hrhc.δ_MAX
    δ_min = hrhc.δ_MIN
    δ_steps = hrhc.δ_STEPS
    
    # 3D array to store motion_primitives
    motion_primitives = zeros(v_steps,2*δ_steps+1,2) # v: 0,...,99; δ: -45:45, (arc length, +/- radius)

    v = linspace(v_min,v_max,v_steps)*ones(1,2*δ_steps+1)
    δ = (linspace(-δ_max,δ_max,δ_steps*2+1)*ones(1,v_steps))' # steering angle

    motion_primitives[:,:,1] = v*Δt*h # arc length = velocity * time
    motion_primitives[:,:,2] = car_length./sin(δ) # radius of curvature (+ or -)
    motion_primitives[:,1+δ_steps,2] = Inf; # radius of curvature is infinite if steering angle = 0
    
    destination_primitives = zeros(v_steps,2*δ_steps+1,h,3) # lookup table defining car's location at each of next h time steps

    for i = 1:h
        # angle = 2π * arc_length / r
        dθ = v*Δt*i ./ motion_primitives[:,:,2]
        
        # dX = abs(radius) * sin(angle)
        destination_primitives[:,:,i,1] = abs(motion_primitives[:,:,2]) .* sin(abs(dθ))
        destination_primitives[:,1+δ_steps,i,1] = v[:,1+δ_steps]*Δt*i # centerline
        
        # dY = radius * (1 - cos(angle))
        destination_primitives[:,:,i,2] = motion_primitives[:,:,2].*(1 - cos(dθ))
        destination_primitives[:,1+δ_steps,i,2] = 0 # centerline
        
        destination_primitives[:,:,i,3] = dθ
    end
    
    # motion_primitives[v, δ, 1,2] = s (arc_length), r (radius of curvature)
    # destination_primitives are index by [v, δ, h, 1,2,3] = dx, dy, dθ = changes in x, y and θ after h time steps
    return v, δ, motion_primitives, destination_primitives

end

MotionPrimitives (generic function with 1 method)

In [4]:
# quadratic logical mask
function QuadraticMask!(library)
    A = size(library)[1]
    B = size(library)[2]
    X = linspace(0,99,A)*ones(1,B)
    Y = ones(A,1)*linspace(-20,20,B)'
    f = 24*Y.^2 + X.^2
    mask = 1.0*(f.<10000)
    
#     fig = figure()
#     ax = gca()
#     p = scatter(X,Y.*mask)
#     xlabel("velocity")
#     ylabel("steering angle")
#     fig[:canvas][:draw]()

    if length(size(library)) == 3
        for i in 1 : size(library)[3]
            library[:,:,i] = library[:,:,i] .* mask
        end
    end
    if length(size(library)) == 4
        for i in 1 : size(library)[3]
            for j in 1 : size(library)[4]
                library[:,:,i,j] = library[:,:,i,j] .* mask
            end
        end
    end
end

QuadraticMask! (generic function with 1 method)

# HRHC Algorithm

### 0. Initialize motion primitives

### 1. Identify eligible (reachable) motion primitives based on current state

In [5]:
function getLegalMoves(hrhc::HRHC,ΔXYθ, v_commands, δ_commands)
    v_norm = hrhc.v / hrhc.V_MAX

    # Restrict search space to reachable states
    v_index_low = max(1, round(Int,(v_norm - hrhc.ΔV₋)*hrhc.V_STEPS)) # index of lowest reachable v in the next time step
    v_index_high = min(hrhc.V_STEPS, round(Int, (v_norm + hrhc.ΔV₊)*hrhc.V_STEPS)) # highest reachable v in the next time step

    # Restrict search space to reachable states
    δ_index_low = max(1, (hrhc.δ_STEPS+1) + round(Int,((hrhc.δ - hrhc.Δδ)/(hrhc.δ_MAX - hrhc.δ_MIN))*(2*hrhc.δ_STEPS+1)))
    δ_index_high = min((2*hrhc.δ_STEPS+1), (hrhc.δ_STEPS+1) + round(Int,((hrhc.δ + hrhc.Δδ)/(hrhc.δ_MAX - hrhc.δ_MIN))*(2*hrhc.δ_STEPS+1)))

    # legal_moves = motion_primitives[v_index_low:v_index_high,δ_index_low:δ_index_high,:]
    legal_ΔXYθ = ΔXYθ[v_index_low:v_index_high,δ_index_low:δ_index_high,hrhc.h,:] # ΔX, ΔY, Δθ
    legal_v = v_commands[v_index_low:v_index_high,δ_index_low:δ_index_high]
    legal_δ = δ_commands[v_index_low:v_index_high,δ_index_low:δ_index_high]
    
    return legal_ΔXYθ, legal_v, legal_δ, v_index_low, v_index_high, δ_index_low, δ_index_high
end

getLegalMoves (generic function with 1 method)

### 2. Calculate resulting final state (posG) from applying each given motion primitive

In [6]:
function getSuccessorStates(state, legal_ΔXYθ)
    pos = state.posG # global x,y,z

    ΔX = legal_ΔXYθ[:,:,1] * cos(pos.θ) + legal_ΔXYθ[:,:,2] * -sin(pos.θ)
    ΔY = legal_ΔXYθ[:,:,1] * sin(pos.θ) + legal_ΔXYθ[:,:,2] * cos(pos.θ)
    Δθ = legal_ΔXYθ[:,:,3]

    successor_states = zeros(size(legal_ΔXYθ))
    successor_states[:,:,1] = ΔX + pos.x
    successor_states[:,:,2] = ΔY + pos.y
    successor_states[:,:,3] = Δθ + pos.θ
    
    return successor_states
end

getSuccessorStates (generic function with 1 method)

In [7]:
# PyPlot.axis("equal")
# PyPlot.scatter(successor_states[:,:,1],successor_states[:,:,2])

### 3. Project posG onto track to determine if a given motion is legal (in bounds)

In [8]:
function projectToCenterline(successor_states, roadway)
    s = zeros(size(successor_states[:,:,1]))
    t = zeros(size(s))
    ϕ = zeros(size(s))
    tag = zeros(size(s))

    successor_states_grid = reshape(successor_states, length(successor_states[:,:,1]),3) # must reshape to index correctly

    for i in 1 : length(s)
        myProj = Vec.proj(VecSE2(successor_states_grid[i,:]), roadway) # project point onto roadway
        curve = myProj.curveproj
        #     @show myProj.tag # tag determines which segment the car is on (effects the value of s)
        tag[i] = myProj.tag.segment
        s[i] = curve.ind.i + curve.ind.t # total distance traveled
        t[i] = curve.t # distance from centerline
        ϕ[i] = curve.ϕ
    end
    
    # THIS FUNCTION WILL NEED TO BE OVERHAULED for a more robust controller later
    
    # account for changes in roadway segment
    if maximum(tag) == length(roadway.segments) && minimum(tag) == 1
        tag_filter = (tag .== minimum(tag))
    else
        tag_filter = (tag .== maximum(tag))
    end
    s += maximum(s).*tag_filter
    
    
    return s,t,ϕ
end

projectToCenterline (generic function with 1 method)

### 4. Select legal motion primitive that yields highest progress

In [9]:
function selectBestPath(hrhc, roadway, s, t, ϕ, v_index_low, v_index_high, δ_index_low, δ_index_high)
    # will need roadway in order to take care of different lane segments
    t_filter = (abs(t) .< hrhc.T_MAX)
    if sum(t_filter) > 0
        eligible_s = s .* t_filter # filter out bad t values
    end

    index = indmax(eligible_s) # find get a better method of optimizing this
    
    cmd = ind2sub(s,index)
    abs_cmd = (cmd[1]+v_index_low-1, cmd[2]+δ_index_low-1)
    return abs_cmd
end

selectBestPath (generic function with 1 method)

### 5. Check each time step up to horizon to make sure the entire trajectory is legal

In [10]:
function computeTrajectory(cmd_index, ΔXYθ, state)
    pos = state.posG
    
    traj_ΔXYθ = ΔXYθ[cmd_index[1],cmd_index[2],:,:]
    
    ΔX = traj_ΔXYθ[:,1] * cos(pos.θ) + traj_ΔXYθ[:,2] * -sin(pos.θ)
    ΔY = traj_ΔXYθ[:,1] * sin(pos.θ) + traj_ΔXYθ[:,2] * cos(pos.θ)
    Δθ = traj_ΔXYθ[:,3]

    trajectory = zeros(size(traj_ΔXYθ))
    trajectory[:,1] = ΔX + pos.x
    trajectory[:,2] = ΔY + pos.y
    trajectory[:,3] = Δθ + pos.θ
    
    return trajectory
end

computeTrajectory (generic function with 1 method)

In [11]:
function screenTrajectory(trajectory, roadway, hrhc)
    out_of_bounds = false
    
    for i in 1 : size(trajectory,1)
        myProj = Vec.proj(VecSE2(trajectory[i,:]), roadway) # project point onto roadway
        t = myProj.curveproj.t
        if abs(t) > hrhc.T_MAX
            print("T OUT OF BOUNDS!!!")
            out_of_bounds=true
            return out_of_bounds
        end
    end
    return out_of_bounds
end

screenTrajectory (generic function with 1 method)

### 6. Apply the motion primitive for a single time step

In [12]:
# next_state = VehicleState(VecSE2(trajectory[1,:]),roadway,v_commands[abs_cmd[1], abs_cmd[2]])
# action = NextState(next_state)

### 7. Go back to 1

# get HRHC action function

In [13]:
function get_HRHC_action(car::HRHC, v_commands, δ_commands, arc_angle_library, ΔXYθ, scene)
    # get current state
    state = scene.vehicles[1].state
    
    v = state.v # current v
    car.v = v
    
    # get legal moves (reachable from current v, δ)
    legal_ΔXYθ, legal_v, legal_δ, v_id_lo, v_id_hi, δ_id_lo, δ_id_hi = getLegalMoves(car, ΔXYθ, v_commands, δ_commands)
    
    # calculate successor states
    successor_states = getSuccessorStates(state,legal_ΔXYθ)
    
    # project successor states onto track to see if they are legal
    s,t,ϕ = projectToCenterline(successor_states, roadway)
    
    # select command expected to yield highest forward progress
    abs_cmd = selectBestPath(car, roadway, s, t, ϕ,v_id_lo, v_id_hi, δ_id_lo, δ_id_hi)    
    
    # compute trajectory
    trajectory  = computeTrajectory(abs_cmd, ΔXYθ, state)

    # screen trajectory for collisions / validity
    out_of_bounds = screenTrajectory(trajectory, roadway, car)    
    
    next_state = VehicleState(VecSE2(trajectory[1,:]),roadway,v_commands[abs_cmd[1], abs_cmd[2]])
    action = NextState(next_state)
    
    
#     push!(state_history, state)
#     push!(δ_history, car.δ)
#     push!(cmd_history, abs_cmd)
    
    car.δ = δ_commands[abs_cmd[1], abs_cmd[2]]
    car.v = v_commands[abs_cmd[1], abs_cmd[2]]
    
    return action
    
end

get_HRHC_action (generic function with 1 method)

## Initialize Parameters

In [14]:
# scene parameters
scene = Scene()
framerate = 24
context = IntegratedContinuous(1/framerate,3)

models = Dict{Int, DriverModel}()
carcolors = Dict{Int, Colorant}()
base_speed= 20.0

#car parameters
car_length = 4.8 # front wheel to back wheel
car_width = 2.5
v⁰  = 0.0 # initial velocity
δ⁰ = 0.0 # initical steering angle

#controller parameters
h = 10 # look-ahead horizon
Δt = 1.0/framerate # time-step size
V_MIN = 0.0 # mininmum velocity (m/s)
V_MAX = 100.0 # maximum velocity (m/s)
V_STEPS = 101 # resolution of velocity
δ_MAX = Float64(π)/8 # max steering angle (radians)
δ_MIN = -δ_MAX # min steering angle (radians)
δ_STEPS = 16 # resolution of steering angle
ΔV₊ = 1.55 / V_MAX
ΔV₋ = 3.05 / V_MAX
Δδ = Float64(π)/16

#track parameters
lane_width = 4.0*DEFAULT_LANE_WIDTH
radius = 45.0
edge_buffer = 0.25
T_MAX = lane_width/2.0 - car_width/2.0 - edge_buffer # max allowable projection distance from center of lane
obstacles = Set()

#initialize track
srand(0)

roadway = gen_stadium_roadway(1, radius=radius, lane_width=lane_width)
roadind = RoadIndex(proj(VecSE2(0.0,-1*DEFAULT_LANE_WIDTH,0.0), roadway))

vehstate = VehicleState(Frenet(roadind, roadway), roadway, base_speed+randn())
vehdef = VehicleDef(1,AgentClass.CAR, 4.8, 1.8)
push!(scene,Vehicle(vehstate, vehdef))
models[1] = Tim2DDriver(context, rec=SceneRecord(1,context.Δt, 1))
set_desired_speed!(models[1], 30.0 + 20*rand())
carcolors[1] = colorant"red"

actions = Array(DriveAction, length(scene))

# initialize HRHC
myHRHC = HRHC(car_length,car_width,v⁰,δ⁰,h,Δt,V_MIN,V_MAX,V_STEPS,δ_MAX,δ_MIN,δ_STEPS,ΔV₊,ΔV₋,Δδ,T_MAX,obstacles)

# get motion primitives
v_commands, δ_commands, arc_angle_library, ΔXYθ = MotionPrimitives(myHRHC)
QuadraticMask!(arc_angle_library) # eliminate infeasible actions with mask
QuadraticMask!(ΔXYθ)

# Tracking 
state_history = []
δ_history = []
cmd_history = []

0-element Array{Any,1}

# Run HRHC:

In [15]:
function drawsim(t, dt=NaN)
    actions[1] = get_HRHC_action(myHRHC, v_commands, δ_commands, arc_angle_library, ΔXYθ, scene)
    tick!(scene, roadway, actions, models)
    render(scene, roadway, cam=FitToContentCamera(), car_colors=carcolors)
end

drawsim (generic function with 2 methods)

In [16]:
using Reel
film = roll(drawsim, fps=framerate, duration=5)
write("HRHC.gif", film) # Write to a gif file

T OUT OF BOUNDS!!!T OUT OF BOUNDS!!!T OUT OF BOUNDS!!!T OUT OF BOUNDS!!!T OUT OF BOUNDS!!!T OUT OF BOUNDS!!!T OUT OF BOUNDS!!!T OUT OF BOUNDS!!!T OUT OF BOUNDS!!!T OUT OF BOUNDS!!!T OUT OF BOUNDS!!!T OUT OF BOUNDS!!!T OUT OF BOUNDS!!!T OUT OF BOUNDS!!!T OUT OF BOUNDS!!!T OUT OF BOUNDS!!!T OUT OF BOUNDS!!!T OUT OF BOUNDS!!!T OUT OF BOUNDS!!!T OUT OF BOUNDS!!!T OUT OF BOUNDS!!!T OUT OF BOUNDS!!!

"HRHC.gif"

In [ ]:
ticks = fps(framerate)
timestamps = map(_ -> time(), ticks)
map(drawsim, timestamps)

# Testing

### laneTagMap[lane_tag] -> s

In [ ]:
myProj = Vec.proj(VecSE2([-20,0,2]), roadway) # project point onto roadway
# curve = myProj.curveproj

In [ ]:
myProj.tag.segment

In [ ]:
fieldnames(myProj.curveproj)

In [ ]:
myProj.curveproj

In [ ]:
fieldnames(roadway.segments[1].lanes[1].curve[1])

In [ ]:
roadway.segments[1].lanes[1].exits[1].mylane.t

In [ ]:
roadway.segments[3].lanes[1].curve[1]

In [ ]:
roadway.segments[4].lanes[1].curve[end]

In [ ]:
roadMap = Dict()
roadMap[1] = Dict()
roadMap[2] = Dict()

In [ ]:
CurvePt

In [ ]:
roadway.segments[4].lanes[1].curve

In [ ]:
function genRoadProgressMap(roadway)
    roadMap = Dict()
    d = 0
    pt1 = (0,0)
    for segment in roadway.segments
        roadMap[segment.id] = Dict()
        for i in length(segment.lanes[1].curve)
            d = sqrt((pt1[1] - pt2)^2 + ()^2)
            roadMap[segment.id][i] = ()
        end
    end
    
end

## Visualization stuff

In [ ]:
# Visualize motion primitives
xgrid = linspace(V_MIN,V_MAX,V_STEPS)*ones(1,2*δ_STEPS+1)
ygrid = (linspace(-δ_MAX,δ_MAX,δ_STEPS*2+1)*ones(1,V_STEPS))' # steering angle
z = ΔXYθ[:,:,10,1]

PyPlot.plot_surface(xgrid, ygrid, z, rstride=1,edgecolors="k", cstride=1,
    alpha=0.8, linewidth=0.25)

In [ ]:
car = myHRHC

In [ ]:
i = length(state_history)-10

In [ ]:
# state = VehicleState(VecSE2(0,0,Float64(π)/4), roadway, 20.0)
state = state_history[i]

# @show i = i+1
car.δ = δ_history[i]
car.v = state.v
@show car.v 
@show car.δ
# @show state_history[i].posG.θ
@show state.posG.θ

# v = state_history[i].v # current v
# car.v = v

# get legal moves (reachable from current v, δ)
legal_ΔXYθ, legal_v, legal_δ, v_id_lo, v_id_hi, δ_id_lo, δ_id_hi = getLegalMoves(car, ΔXYθ, v_commands, δ_commands)
@show legal_v[:,1]

@show legal_δ[1,:]

# calculate successor states
# successor_states = getSuccessorStates(state_history[i],legal_ΔXYθ)
successor_states = getSuccessorStates(state,legal_ΔXYθ)

# PyPlot.axis("equal")
# PyPlot.scatter(successor_states[:,:,1],successor_states[:,:,2])

# project successor states onto track to see if they are legal
s,t,ϕ,tag = projectToCenterline(successor_states, roadway)
# PyPlot.axis("equal")
PyPlot.scatter(s,t)
PyPlot.scatter(s[indmax(s)],t[indmax(s)],color="red")
@show indmax(s)

In [ ]:
tag

In [ ]:
if maximum(tag) == 4 && minimum(tag) == 1
    tag_filter = (tag .== minimum(tag))
else
    tag_filter = (tag .== maximum(tag))
end
s += maximum(s).*tag_filter
s

In [ ]:
tag_filter

In [ ]:
tag

In [ ]:
# successor_states[:,6:12,:]

In [ ]:
# legal_δ[:,:]

In [ ]:
# legal_ΔXYθ[:,:,:]

In [ ]:
# select command expected to yield highest forward progress
abs_cmd = selectBestPath(car, roadway, s, t, ϕ,v_id_lo, v_id_hi, δ_id_lo, δ_id_hi)  
@show abs_cmd
@show v_commands[abs_cmd[1],abs_cmd[2]], δ_commands[abs_cmd[1],abs_cmd[2]]

# @show cmd_history[i]
# @show v_commands[cmd_history[i][1],cmd_history[i][2]], δ_commands[cmd_history[i][1],cmd_history[i][2]]

# compute trajectory
trajectory  = computeTrajectory(abs_cmd, ΔXYθ, state_history[i])
PyPlot.axis("equal")
PyPlot.scatter(successor_states[:,:,1],successor_states[:,:,2])
PyPlot.scatter(trajectory[h,1], trajectory[h,2],color="red")

# Features to add

In [ ]:
"""
##########
RoadwayTagMap[lane_tag] -> base distance around the track

##########
Optimization Objective Function
- reward progress on track
- penalize heuristic functions of ϕ, t
--- f(ϕ,t) to penalize steering away from centerline?

##########
ProximityKernel(): gives proximity to a given location against a set of locations that represent obstacles
proximity = ProximityKernel(X,Y,θ,(ObstacleCoords))
proximity = ProximityKernel(s,t,ϕ,(ObstaclesCoords))

##########
Array-based operations for a faster and more robust search through eligible motion primitives
- Search highest V candidates first
- break out of a candidate trajectory as soon as ineligible
- If no trajectory is feasible, restart full search at horizon level h-1


Cognitive hierarchy - ditribution over logit-level K


"""

In [ ]:
"""
Email update:
What I did
What I would like to do
Issues?
"""
